In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datasets import load_dataset

from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, GenerationConfig
import pandas as pd

In [3]:
# Загрузка предобученной модели и токенизатора
model_name = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
tokenizer.add_special_tokens({'pad_token': '<pad>', 'bos_token': '<beginofstring>', 'eos_token': '<endofstring>'})
tokenizer.add_tokens(["<p-1>", "<p-2>"])

2

In [6]:
df = pd.read_table('/kaggle/input/toloka-persona-chat-rus/dialogues.tsv')

In [7]:
df["dialogue"][2]

'<span class=participant_1>Пользователь 1: Привет</span><br /><span class=participant_1>Пользователь 1: Как дела ?</span><br /><span class=participant_2>Пользователь 2: Добрый день!</span><br /><span class=participant_2>Пользователь 2: Хорошо,  чем увлекаетесь?</span><br /><span class=participant_1>Пользователь 1: Я бегаю по утрам а ты?</span><br /><span class=participant_1>Пользователь 1: Есть любимые вещи или еда ?</span><br /><span class=participant_1>Пользователь 1: Занят ?</span><br /><span class=participant_2>Пользователь 2: Я люблю петь в караоке)</span><br /><span class=participant_1>Пользователь 1: Круто )</span><br /><span class=participant_2>Пользователь 2: Люблю готовить пасту,  у меня классно получается!</span><br /><span class=participant_2>Пользователь 2: Любишь готовить?</span><br /><span class=participant_1>Пользователь 1: Это хорошо</span><br /><span class=participant_1>Пользователь 1: Я не эксперт</span><br /><span class=participant_1>Пользователь 1: Я люблю есть арб

In [9]:
def parse_dialogues(dialogue: str) -> list:
    lines = dialogue.split('<br />')
    result = ""
    for i, line in enumerate(lines):
        if '<span class=participant_1' in line:
            msg = line.split(':')[1].split("</span>")[0].strip()
            result+=f"<p-1> {msg} "
        elif '<span class=participant_2' in line:
            msg = line.split(':')[1].split("</span>")[0].strip()
            result+=f"<p-2> {msg} "
    return result

In [11]:
dialogue_data = []
for dialogue in df["dialogue"]:
    dialogue_data.append(parse_dialogues(dialogue))

In [12]:
dialogue_data[2]

'<p-1> Привет <p-1> Как дела ? <p-2> Добрый день! <p-2> Хорошо,  чем увлекаетесь? <p-1> Я бегаю по утрам а ты? <p-1> Есть любимые вещи или еда ? <p-1> Занят ? <p-2> Я люблю петь в караоке) <p-1> Круто ) <p-2> Люблю готовить пасту,  у меня классно получается! <p-2> Любишь готовить? <p-1> Это хорошо <p-1> Я не эксперт <p-1> Я люблю есть арбуз '

In [13]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data[idx]
        encoding = self.tokenizer(
            source_text,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten().cuda(),
            "attention_mask": encoding["attention_mask"].flatten().cuda(),
            }

In [15]:
train_data = dialogue_data[:9000]
val_data = dialogue_data[9000:]

In [16]:
train_dataset = MyDataset(train_data, tokenizer)
val_dataset = MyDataset(val_data, tokenizer)

In [19]:
from tqdm import tqdm

In [84]:
!pip install -U nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [86]:
from nltk.translate.bleu_score import corpus_bleu


In [22]:
# Define function to calculate BLEU score
def calculate_bleu_score(predictions, references):
    return corpus_bleu([[ref] for ref in references], predictions)

In [32]:
def train(train_dataset, model, tokenizer, epochs=3, batch_size=8):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in progress_bar:
            optimizer.zero_grad()
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = input_ids.clone()
            labels[attention_mask == 0] = -100  # ignore padding tokens
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({"Loss": loss.item()})
        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Average Train Loss: {avg_train_loss:.4f}")
    

In [90]:
def val(val_dataset, model, tokenizer, batch_size=8):
    # Валидация
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc=f"Val:")
        for batch in progress_bar:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = input_ids.clone()
            labels[attention_mask == 0] = -100  # игнорируем токены-заполнители
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, 
                                    max_new_tokens=512, pad_token_id=tokenizer.eos_token_id)
            # Декодирование результатов
            for output_ids in outputs:
                decoded_output = tokenizer.decode(output_ids.cpu().numpy(), skip_special_tokens=True)
                predictions.append(decoded_output)
            
            # Декодирование меток
            for label_ids in labels:
                decoded_labels = tokenizer.decode([token_id for token_id in label_ids if token_id != -100], skip_special_tokens=True)
                references.append(decoded_labels)

    bleu_score = calculate_bleu_score(predictions, references)
    print(f"BLEU: {bleu_score:.4f}")


In [92]:
train(train_dataset, model, tokenizer)

Epoch 1/3: 100%|██████████| 1125/1125 [11:59<00:00,  1.56it/s, Loss=3.73]


Epoch 1/3, Average Train Loss: 3.7149


Epoch 2/3: 100%|██████████| 1125/1125 [11:59<00:00,  1.56it/s, Loss=3.61]


Epoch 2/3, Average Train Loss: 4.6452


Epoch 3/3: 100%|██████████| 1125/1125 [11:59<00:00,  1.56it/s, Loss=3.03]

Epoch 3/3, Average Train Loss: 3.1542


In [98]:
val(val_dataset, model, tokenizer)

Val:: 100%|██████████| 127/127 [12:36<00:00,  5.96s/it]


BLEU: 0.3320


In [108]:
generation_config = GenerationConfig(
    max_length=200,
    temperature=0.8,
    do_sample=True,
    repetition_penalty=1.2,
    top_p=0.8
)

In [109]:
def answer(q):
    model_inputs = tokenizer([q], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, generation_config=generation_config)
    return tokenizer.decode(generated_ids[0], Skip_special_tokens= True)

In [110]:
def chat():
    current_dialogue = ""
    while True:
        user_input = input("USER: ")
        if user_input == "end":
            break
        current_dialogue += f" <p-1> {user_input} <p-2>"
        ans = answer(current_dialogue)[len(current_dialogue):].split("<")[0]
        current_dialogue += ans
        print("BOT:", ans)

In [111]:
chat()

USER:  Привет


BOT:  Как дела? 


USER:  Нормально у тебя как?


BOT:  Хорошо, а ты чем занимаешься? 


USER:  Отдыхаю, а ты?


BOT:  Оо. Я сейчас на работе) люблю читать и гулять 


USER:  Я тоже люблю читать. Что читаешь?


BOT:  Читаю книгу про Гарри Поттера. 


KeyboardInterrupt: Interrupted by user

In [112]:
chat()

USER:  Привет


BOT:  Я работаю тренером в больнице. Ты любишь спорт? 


USER:  Не очень.


BOT:  Да, люблю играть со своим хомяком и плавать с ним) а ты чем занимаешься 


USER:  Сижу за компьютером.


...

USER:  А я люблю читать.


BOT:  У тебя есть хобби? 


USER:  Да. У тебя какие хобби?


BOT:  Круто! 


KeyboardInterrupt: Interrupted by user

In [113]:
chat()

USER:  Привет.


BOT:  Как тебя зовут? 


USER:  Камиль, а тебя как?


BOT:  Меня Сергей) а ты откуда? 


USER:  С Челябинска, а ты?


...

USER:  Приятно познакомиться! Чем занимаешься?


BOT:  А я сейчас собираюсь в бассейн, не смотря на то что живу за границей 


USER:  Круто! А я читаю книгу.


BOT:  И ещё люблю готовить. Особенно супы. Но все равно приходится много читать 


KeyboardInterrupt: Interrupted by user